In [1]:
import os
from langchain.document_loaders import TextLoader

import warnings
warnings.filterwarnings("ignore")

############################################################################
## TextLoader
############################################################################
#loader = TextLoader('little.prince.txt')
#pages = loader.load()

from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("./data/2024.kbo.rules.pdf")
pages = loader.load()

text = pages[0].page_content
text

''

In [2]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

############################################################################
## TextSplitter
############################################################################
# 문서를 문장으로 분리
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
)
docs = text_splitter.split_documents(pages)
print(docs[0])
print(docs[1])

page_content='2024년도
KBO 리그 규정' metadata={'source': './data/2024.kbo.rules.pdf', 'file_path': './data/2024.kbo.rules.pdf', 'page': 4, 'total_pages': 102, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'nPDF (pdftk 1.41)', 'producer': '3-Heights™ PDF Optimization Shell 6.3.1.5 (http://www.pdf-tools.com)', 'creationDate': "D:20240403201030-09'00'", 'modDate': 'D:20240404055647Z', 'trapped': ''}
page_content='목 차
제1장 KBO 정규시즌
제1조
경기방식·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
 1
제2조
승률계산법·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
 1
제3조
연도 구단순위 및 기록·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
·
 1
제4조
경기규칙·
·
·
·
·' metadata={'source': './data/2024.kbo.rules.pd

In [3]:
# 문장 임베딩 및 벡터 저장소 생성
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
############################################################################
## Text Vector Embedding
############################################################################
# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    #model_name='jhgan/ko-sroberta-nli',
    model_name='BAAI/bge-m3',
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},
    # model_kwargs={'device':'mps'},
    encode_kwargs={'normalize_embeddings':True},
)


############################################################################
## VectorStore(Chroma)
############################################################################
vectorstore = Chroma.from_documents(docs, embeddings)


# 벡터 저장소 경로 설정
vectorstore_path = 'vectorstore'
os.makedirs(vectorstore_path, exist_ok=True)

# 벡터 저장소 생성 및 저장
vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
vectorstore.persist()
print("Vectorstore created and persisted")


Vectorstore created and persisted


In [6]:
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama

############################################################################
## retriever
############################################################################
model = ChatOllama(model="llama3.1:latest", temperature=0)

# 검색 쿼리
query = "승률계산법을 알려 줘"

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

docs = retriever.get_relevant_documents(query)
print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)


3
투수 부문 - 평균자책점상, 승리상
타자 부문 - 타율상, 홈런상, 타점상을
북부, 남부리그로 나누어 시상한다.
(공식야구규칙 9.22에 의거 선정)
{'author': '', 'creationDate': "D:20240403201030-09'00'", 'creator': 'nPDF (pdftk 1.41)', 'file_path': './data/2024.kbo.rules.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': 'D:20240404055647Z', 'page': 59, 'producer': '3-Heights™ PDF Optimization Shell 6.3.1.5 (http://www.pdf-tools.com)', 'source': './data/2024.kbo.rules.pdf', 'subject': '', 'title': '', 'total_pages': 102, 'trapped': ''}


In [7]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

############################################################################
## Generate RAG Chain
############################################################################

# Prompt
template = '''Answer the question in korean based only on the following context:
{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Chain 실행
query = "승률계산법을 알려 줘"
answer = rag_chain.invoke(query)

print("Answer:", answer)

Answer: 제2조 승률계산법에 따르면, 승률은 승수/(승수＋패수)로 계산합니다.
